# get geo

In [1]:
import pandas as pd
import numpy as np

In [2]:
import geo_getter as get

## urban villages

In [3]:
bg_10 = pd.read_csv('../data/geo/bgp_vil_10.csv')
bg_20 = pd.read_csv('../data/geo/bgp_vil_20.csv')

In [4]:
uv10 = bg_10[['name','land_acre','aland10']].copy()
uv10 = uv10.groupby(['name']).sum().reset_index()
uv10.rename(columns={'land_acre':'AREALAND_ACRE10','aland10':'AREALAND10'},\
                        inplace=True)

In [5]:
uv20 = bg_20[['name','land_acre','aland20']].copy()
uv20 = uv20.groupby(['name']).sum().reset_index()
uv20.rename(columns={'land_acre':'AREALAND_ACRE20','aland20':'AREALAND20'},\
                        inplace=True)

In [6]:
uv = pd.merge(uv10,uv20,how='left',on='name')

## cities

In [7]:
cities = {455000:'Phoenix',473000:'Tempe',465000:'Scottsdale',427820:'Glendale',\
          4865000:'San Antonio', 4819000:'Dallas',1235000:'Jacksonville',\
          1836000:'Indianapolis'}

geoids = list(cities.keys())

In [8]:
pl10 = get.make_city('2010')
pl20 = get.make_city('2020')

In [9]:
places = pd.merge(pl10,pl20,how='left',on='GEOID')

In [10]:
places = places[places.GEOID.isin(list(cities.keys()))].copy()

In [11]:
places.rename(columns={'BASENAME10':'name'},inplace=True)
places = places.drop(['GEOID','AREAWATER10','BASENAME20','AREAWATER20'],axis=1)

## maricopa

In [13]:
mar10 = get.make_maricopa('2010')
mar20 = get.make_maricopa('2020')

In [14]:
mar = pd.merge(mar10,mar20,how='left',on='GEOID')

In [15]:
mar.rename(columns={'BASENAME10':'name'},inplace=True)
mar = mar.drop(['GEOID','AREAWATER10','BASENAME20','AREAWATER20'],axis=1)

## arizona

In [17]:
az10 = get.make_az('2010')
az20 = get.make_az('2020')
az = pd.merge(az10,az20,how='left',on='GEOID')
az['name'] = 'AZ'

In [18]:
az = az.drop(['GEOID','AREAWATER10','BASENAME10','BASENAME20','AREAWATER20'],axis=1)

## us

In [20]:
us10 = get.make_us('2010')
us20 = get.make_us('2020')
us = pd.merge(us10,us20,how='left',on='FUNCSTAT')

In [21]:
us['name'] = 'US'
us = us.drop(['FUNCSTAT','AREAWATER10','AREAWATER20'],axis=1)

## make big geography table - no urban vil

In [23]:
geos = pd.concat([uv,places,mar,az,us])

## read in pop and housing from output folder

In [24]:
data = pd.read_csv('output/dec_pop_hou_race.csv')

In [26]:
data = data[['name','Hou_10E','Hou_20E','Pop_10E','Pop_20E']]

## calc housing and pop density in 2010 and 2020

In [27]:
geos = pd.merge(geos,data,how='left',on='name')

In [29]:
geos['HouDen_10'] = geos.Hou_10E / geos.AREALAND_ACRE10
geos['PopDen_10'] = geos.Pop_10E / geos.AREALAND_ACRE10
geos['HouDen_20'] = geos.Hou_20E / geos.AREALAND_ACRE20
geos['PopDen_20'] = geos.Pop_20E / geos.AREALAND_ACRE20

In [30]:
geos

,name,AREALAND_ACRE10,AREALAND10,AREALAND_ACRE20,AREALAND20,Hou_10E,Hou_20E,Pop_10E,Pop_20E,HouDen_10,PopDen_10,HouDen_20,PopDen_20
0,Ahwatukee Foothills,2.991293e+04,121053509,2.984415e+04,1.207752e+08,33528,34527,77344,80604,1.120853,2.585638,1.156910,2.700831
1,Alhambra,1.241612e+04,50246329,1.249870e+04,5.058052e+07,52066,52997,126444,144882,4.193420,10.183858,4.240202,11.591767
2,Camelback East,2.334777e+04,94485200,2.411085e+04,9.757328e+07,69263,75182,132153,144949,2.966579,5.660199,3.118182,6.011776
3,Central City,1.537227e+04,62209450,1.427675e+04,5.777604e+07,23571,26537,59259,61452,1.533346,3.854929,1.858756,4.304341
4,Deer Valley,3.745064e+04,151557602,3.774143e+04,1.527344e+08,71285,76899,167302,185789,1.903439,4.467267,2.037522,4.922681
5,Desert View,4.968658e+04,201074752,4.765469e+04,1.928520e+08,23188,29184,51817,65007,0.466685,1.042877,0.612406,1.364126
6,Encanto,6.945658e+03,28108125,6.877950e+03,2.783412e+07,28214,30714,55934,60015,4.062106,8.053088,4.465575,8.725711
7,Estrella,2.855373e+04,115553025,2.969180e+04,1.201586e+08,23595,25571,84500,96125,0.826337,2.959333,0.861214,3.237425
8,Laveen,1.628645e+04,65909031,1.555667e+04,6.295570e+07,15884,18677,47834,66047,0.975289,2.937043,1.200578,4.245575
9,Maryvale,2.654606e+04,107428274,2.680351e+04,1.084701e+08,66024,67598,210352,229496,2.487148,7.924037,2.521983,8.562161
